In [ ]:
pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PolynomialFeatures, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, chi2, RFE
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from scipy.stats import uniform, randint
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.pipeline import Pipeline as IMBPipeline
import tensorflow as tf
from tensorflow import keras

In [ ]:
import warnings

# Filter out all warnings
warnings.filterwarnings("ignore")

In [ ]:
file_path = '/content/sample_data/classdata_1.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print("First few rows of the dataset:")
data.head()

First few rows of the dataset:


,G,C,B,A,Va,Vb,Vc,Ia,Ib,Ic,Fault Location
0,0,0,0,0,125.610808,-132.965993,7.355185,0.098887,-0.074821,-0.024065,100
1,0,0,0,0,124.148556,-134.194944,10.046388,0.098362,-0.076104,-0.022259,100
2,0,0,0,0,122.440300,-135.250998,12.810698,0.097285,-0.077032,-0.020253,100
3,0,0,0,0,120.794470,-136.335386,15.540916,0.096453,-0.078125,-0.018328,100
4,0,0,0,0,119.209350,-137.438542,18.229193,0.095828,-0.079369,-0.016458,100


In [ ]:
print("\nChecking for missing values:")
missing_values = data.isnull().sum()

# Checking for any duplicates
duplicate_rows = data.duplicated().sum()

missing_values,duplicate_rows


Checking for missing values:


(G                 0
 C                 0
 B                 0
 A                 0
  Va               0
  Vb               0
  Vc               0
  Ia               0
  Ib               0
  Ic               0
 Fault Location    0
 dtype: int64,
 0)

In [ ]:
data['Fault_Type'] = data[['G', 'C', 'B', 'A']].astype(str).agg(''.join, axis=1)

# Defining the fault types
fault_types = {
    '0000': 'No Fault',
    '1001': 'Line A to Ground Fault',
    '0110': 'Line B to Line C fault',
    '0111': 'Line A Line B Line C',
    '1111': 'Line A Line B Line C to Ground Fault',
    '1011': 'Line A Line B to Ground Fault',
    '0101': 'Line A to Line C fault',
    '1101': 'Line A Line C to Ground Fault',
    '1100': 'Line C to Ground Fault',
    '1110': 'Line B Line C to Ground Fault',
    '1010': 'Line B to Ground Fault',
    '0011': 'Line A to Line B fault'
}

# Mapping fault type codes to fault type names
data['Fault_Type'] = data['Fault_Type'].map(fault_types)

# Counting the occurrences of each fault type
fault_type_counts = data['Fault_Type'].value_counts()
fault_type_counts

Fault_Type
No Fault                                16008
Line A to Ground Fault                  16008
Line B to Ground Fault                  16008
Line A Line B to Ground Fault           16008
Line A Line C to Ground Fault           16008
Line A Line B Line C to Ground Fault    16008
Line A to Line B fault                  16008
Line B to Line C fault                  16008
Line A Line B Line C                    16008
Line A to Line C fault                  16008
Line C to Ground Fault                  16007
Line B Line C to Ground Fault           16007
Name: count, dtype: int64

In [ ]:
# labelling of data
data['Fault_Type'][data['Fault_Type'] == '0000' ] = 'NO Fault'
data['Fault_Type'][data['Fault_Type'] == '1001' ] = 'Line A to Ground Fault'
data['Fault_Type'][data['Fault_Type'] == '1010' ] = 'Line B to Ground Fault'
data['Fault_Type'][data['Fault_Type'] == '1100' ] = 'Line C to Ground Fault'
data['Fault_Type'][data['Fault_Type'] == '0011' ] = 'Line A to Line B Fault'
data['Fault_Type'][data['Fault_Type'] == '0101' ] = 'Line A to Line C Fault'
data['Fault_Type'][data['Fault_Type'] == '0110' ] = 'Line B to Line C Fault'
data['Fault_Type'][data['Fault_Type'] == '1011' ] = 'Line A Line B to Ground Fault'
data['Fault_Type'][data['Fault_Type'] == '1101' ] = 'Line A Line C to Ground Fault'
data['Fault_Type'][data['Fault_Type'] == '1110' ] = 'Line B Line C to Ground Fault'
data['Fault_Type'][data['Fault_Type'] == '0111' ] = 'Line A Line B Line C'
data['Fault_Type'][data['Fault_Type'] == '1111' ] = 'Line A Line B Line C to Ground Fault'


In [ ]:
#checking out random sample data of size 12.
data.sample(12).style.set_properties(**{'background-color': 'black',
                           'color': 'white',
                           'border-color': 'darkblack'})

,G,C,B,A,Va,Vb,Vc,Ia,Ib,Ic,Fault Location,Fault_Type
140533,0,1,1,1,-27.382939,22.256128,5.126811,-0.703463,-0.916138,1.619602,40,Line A Line B Line C
82298,1,1,0,1,27.400813,40.497777,-32.113313,-0.386869,0.026836,-0.443398,60,Line A Line C to Ground Fault
134058,1,1,1,0,18.707847,-28.086933,5.213505,1.023327,-0.034615,0.010303,40,Line B Line C to Ground Fault
6647,1,1,0,0,39.009495,-116.101745,-45.552039,0.312036,-0.052690,-0.030640,100,Line C to Ground Fault
60091,1,1,1,0,-32.275518,-28.090437,168.119844,0.640344,-0.945243,0.063528,75,Line B Line C to Ground Fault
125485,1,0,1,0,-129.300546,-0.470224,168.319411,-0.045399,-1.314134,0.070145,40,Line B to Ground Fault
178123,1,1,0,1,5.006727,-172.927020,24.192116,1.432167,-0.059008,-0.277966,25,Line A Line C to Ground Fault
59038,1,1,0,1,26.112885,-170.997847,34.888870,0.827883,-0.065645,-0.612044,75,Line A Line C to Ground Fault
32847,1,0,1,1,-142.007743,47.145848,-3.733534,-0.066697,-0.118755,0.813474,80,Line A Line B to Ground Fault
173995,1,0,1,0,91.873541,6.666479,-177.304650,0.039841,1.564878,-0.062754,25,Line B to Ground Fault


In [ ]:
print("\nDescriptive Statistics:")
data.describe()


Descriptive Statistics:


,G,C,B,A,Va,Vb,Vc,Ia,Ib,Ic,Fault Location
count,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000,192094.000000
mean,0.583329,0.583329,0.583334,0.583339,-0.007478,-0.099425,0.107374,-0.044610,-0.036074,0.081035,56.874967
std,0.493009,0.493009,0.493008,0.493007,84.692595,84.683232,84.589003,12.698084,12.811357,3.520637,25.241221
min,0.000000,0.000000,0.000000,0.000000,-181.020144,-181.029327,-181.086373,-175.757809,-177.323333,-47.812587,25.000000
25%,0.000000,0.000000,0.000000,0.000000,-41.064725,-41.089228,-40.823937,-0.319453,-0.309075,-0.109028,28.750000
50%,1.000000,1.000000,1.000000,1.000000,-0.059960,-0.120366,0.202213,-0.006779,-0.005198,0.011558,50.000000
75%,1.000000,1.000000,1.000000,1.000000,41.016818,40.939738,41.098335,0.186773,0.199516,0.352900,78.750000
max,1.000000,1.000000,1.000000,1.000000,181.063438,181.040693,181.006102,175.746319,177.282892,48.279849,100.000000


In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()

In [ ]:
print(data.columns)

Index(['G', 'C', 'B', 'A', ' Va', ' Vb', ' Vc', ' Ia', ' Ib', ' Ic',
       'Fault Location', 'Fault_Type'],
      dtype='object')


In [ ]:
import pandas as pd

# Load your dataset
data = pd.read_csv('/content/sample_data/classdata_1.csv')

data['Fault_Type'] = data[['G', 'C', 'B', 'A']].astype(str).agg(''.join, axis=1)

# Defining the fault types
fault_types = {
    '0000': 'No Fault',
    '1001': 'Line A to Ground Fault',
    '0110': 'Line B to Line C fault',
    '0111': 'Line A Line B Line C',
    '1111': 'Line A Line B Line C to Ground Fault',
    '1011': 'Line A Line B to Ground Fault',
    '0101': 'Line A to Line C fault',
    '1101': 'Line A Line C to Ground Fault',
    '1100': 'Line C to Ground Fault',
    '1110': 'Line B Line C to Ground Fault',
    '1010': 'Line B to Ground Fault',
    '0011': 'Line A to Line B fault'
}

# Mapping fault type codes to fault type names
data['Fault_Type'] = data['Fault_Type'].map(fault_types)

# Save the labeled dataset
data.to_csv('/content/labeled_data.csv', index=False)




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# Load the dataset
dataset = pd.read_csv('/content/labeled_data.csv')

# Split the dataset into train and test sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Create directories for train and test folders if they don't exist
train_dir = 'train'
test_dir = 'test'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Save train and test datasets to train and test folders respectively
train_data.to_csv(os.path.join(train_dir, 'train.csv'), index=False)
test_data.to_csv(os.path.join(test_dir, 'test.csv'), index=False)

print("Data split and saved successfully.")

# Now you can proceed to train your model using the train data and test it using the test data.



Data split and saved successfully.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Adjust your model architecture to accept input with 6 features instead of 10
model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(6, activation='softmax')
])

In [ ]:
# locan
location_model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(6, activation='softmax')
])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load your dataset
data = pd.read_csv('/content/labeled_data.csv')

# Separate features and labels
X = data.drop(columns=['Fault_Type', 'Fault Location', 'G', 'C', 'B', 'A'])
y = data['Fault_Type']  # Assuming 'Fault Type' is the column containing fault types

# Check for non-numeric columns and convert them to numeric
non_numeric_cols = X.select_dtypes(include=['object']).columns
for col in non_numeric_cols:
    X[col] = pd.to_numeric(X[col], errors='coerce')

# Drop rows with NaN values
X.dropna(inplace=True)
y = y[X.index]  # Update labels accordingly

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train.head);
# print(y_train.head);
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Separate features and labels location
X_location = data.drop(columns=['Fault_Type', 'Fault Location', 'G', 'C', 'B', 'A'])
y_location = data['Fault Location']  # Assuming 'Fault Type' is the column containing fault types


# Check for non-numeric columns and convert them to numeric
non_numeric_cols = X_location.select_dtypes(include=['object']).columns
for col in non_numeric_cols:
    X_location[col] = pd.to_numeric(X_location[col], errors='coerce')

# Drop rows with NaN values
X_location.dropna(inplace=True)
y_location = y_location[X_location.index]  # Update labels accordingly

# Split the dataset into training and testing sets
X_location_train, X_location_test, y_location_train, y_location_test = train_test_split(X_location, y_location, test_size=0.2, random_state=42)


# Standardize the features
scaler = StandardScaler()
X_train_location_scaled = scaler.fit_transform(X_location_train)
X_test_location_scaled = scaler.transform(X_location_test)

In [ ]:
# Feature scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [ ]:
# location feture scaling
scaler = MinMaxScaler()
X_location_train_scaled = scaler.fit_transform(X_location_train)
X_location_test_scaled = scaler.transform(X_location_test)

In [ ]:
from tensorflow.keras import models, layers

# Define the FNN model
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Adjust the number of units and activation function as needed
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use appropriate loss function for classification
              metrics=['accuracy'])

# Print model summary
print(model.summary())


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 64)                448       
                                                                 
 dense_51 (Dense)            (None, 64)                4160      
                                                                 
 dense_52 (Dense)            (None, 64)                4160      
                                                                 
 dense_53 (Dense)            (None, 10)                650       
                                                                 
Total params: 9418 (36.79 KB)
Trainable params: 9418 (36.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# location

# Define the FNN model
location_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_location_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # Adjust the number of units and activation function as needed
])

# Compile the model
location_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use appropriate loss function for classification
              metrics=['accuracy'])

# Print model summary
print(location_model.summary())


Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 64)                448       
                                                                 
 dense_55 (Dense)            (None, 64)                4160      
                                                                 
 dense_56 (Dense)            (None, 64)                4160      
                                                                 
 dense_57 (Dense)            (None, 10)                650       
                                                                 
Total params: 9418 (36.79 KB)
Trainable params: 9418 (36.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder on y_train to identify unique fault descriptions
label_encoder.fit(y_train)

# Transform y_train to encoded numeric labels
y_train_encoded = label_encoder.transform(y_train)

# Transform y_test using the same label encoder
y_test_encoded = label_encoder.transform(y_test)

# Print mapping of original fault descriptions to encoded numeric labels
print("Mapping of fault descriptions to numeric labels:")
for label, code in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
    print(f"{label}: {code}")

# Print encoded numeric labels for y_train
print("Encoded labels for y_train:")
print(y_train_encoded.shape)

# Print encoded numeric labels for y_test
print("Encoded labels for y_test:")
print(y_test_encoded)


Mapping of fault descriptions to numeric labels:
Line A Line B Line C: 0
Line A Line B Line C to Ground Fault: 1
Line A Line B to Ground Fault: 2
Line A Line C to Ground Fault: 3
Line A to Ground Fault: 4
Line A to Line B fault: 5
Line A to Line C fault: 6
Line B Line C to Ground Fault: 7
Line B to Ground Fault: 8
Line B to Line C fault: 9
Line C to Ground Fault: 10
No Fault: 11
Encoded labels for y_train:
(153675,)
Encoded labels for y_test:
[0 8 6 ... 8 0 8]


In [ ]:
# location
# Initialize LabelEncoder
location_label_encoder = LabelEncoder()

# Fit LabelEncoder on y_train to identify unique fault descriptions
location_label_encoder.fit(y_location_test)

# Transform y_train to encoded numeric labels
y_location_train_encoded = location_label_encoder.transform(y_location_train)

# Transform y_test using the same label encoder
y_location_test_encoded = location_label_encoder.transform(y_location_test)

# Print mapping of original fault descriptions to encoded numeric labels
print("Mapping of fault descriptions to numeric labels:")
for label, code in zip(location_label_encoder.classes_, location_label_encoder.transform(location_label_encoder.classes_)):
    print(f"{label}: {code}")

# Print encoded numeric labels for y_train
print("Encoded labels for y_train:")
print(y_location_train_encoded.shape)

# Print encoded numeric labels for y_test
print("Encoded labels for y_test:")
print(y_location_test_encoded)


Mapping of fault descriptions to numeric labels:
25: 0
40: 1
50: 2
60: 3
75: 4
80: 5
100: 6
Encoded labels for y_train:
(153675,)
Encoded labels for y_test:
[4 0 4 ... 2 3 4]


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical


# Convert encoded labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=num_classes)


# Define the FNN model
model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer with correct num_classes units and softmax activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
history = model.fit(X_train_scaled, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 64)                448       
                                                                 
 dense_59 (Dense)            (None, 64)                4160      
                                                                 
 dense_60 (Dense)            (None, 64)                4160      
                                                                 
 dense_61 (Dense)            (None, 12)                780       
                                                                 
Total params: 9548 (37.30 KB)
Trainable params: 9548 (37.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
3842/3842 [==============================] - 14s 3ms/step - loss: 0.9443 - accuracy: 0.6558 - val_loss: 0.5869 - val_accuracy: 0.7712
E

In [ ]:
# location
# Convert encoded labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_location_train_one_hot = to_categorical(y_location_train_encoded, num_classes=num_classes)
y_location_test_one_hot = to_categorical(y_location_test_encoded, num_classes=num_classes)

# Define the FNN model
location_model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer with correct num_classes units and softmax activation
])

# Compile the model
location_model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use categorical crossentropy for multi-class classification
              metrics=['accuracy'])

# Print model summary
print(location_model.summary())

# Train the model
location_history = location_model.fit(X_location_train_scaled, y_location_train_one_hot, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_accuracy = location_model.evaluate(X_location_test_scaled, y_location_test_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_62 (Dense)            (None, 64)                448       
                                                                 
 dense_63 (Dense)            (None, 64)                4160      
                                                                 
 dense_64 (Dense)            (None, 64)                4160      
                                                                 
 dense_65 (Dense)            (None, 12)                780       
                                                                 
Total params: 9548 (37.30 KB)
Trainable params: 9548 (37.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/10
3842/3842 [==============================] - 13s 3ms/step - loss: 1.8987 - accuracy: 0.2580 - val_loss: 1.8845 - val_accuracy: 0.2607
E

In [ ]:
print(X_train_scaled.shape[1]);

6


In [ ]:
# Predict on unseen data
new_data = pd.read_csv('/content/test/test.csv')
X_new = new_data.drop(columns=['Fault_Type', 'Fault Location', 'G', 'B', 'C', 'A'])  # Adjust column names as needed
X_new_scaled = scaler.transform(X_new)  # Scale features using the same scaler
predictions = model.predict(X_new_scaled)


1201/1201 [==============================] - 2s 1ms/step


In [ ]:
# location

X_location_new = new_data.drop(columns=['Fault_Type', 'Fault Location', 'G', 'B', 'C', 'A'])  # Adjust column names as needed
X_location_new_scaled = scaler.transform(X_location_new)  # Scale features using the same scaler
predictions = model.predict(X_location_new_scaled)

1201/1201 [==============================] - 2s 1ms/step


In [ ]:
# Define a function to convert predicted probabilities to predicted labels
def get_predicted_labels(probabilities):
    return np.argmax(probabilities, axis=1)

# Make predictions on the test set
predictions_probabilities = model.predict(X_test_scaled)
predicted_labels = get_predicted_labels(predictions_probabilities)

# Decode predicted labels from numeric to original fault descriptions
predicted_faults = label_encoder.inverse_transform(predicted_labels)

# Compare predicted faults with actual faults
comparison_df = pd.DataFrame({'Actual Faults': y_test, 'Predicted Faults': predicted_faults})
print(comparison_df.head())

# Evaluate model performance metrics
from sklearn.metrics import classification_report, confusion_matrix

print("Classification Report:")
print(classification_report(y_test, predicted_faults))

print("Confusion Matrix:")
print(confusion_matrix(y_test, predicted_faults))


1201/1201 [==============================] - 2s 1ms/step
                 Actual Faults               Predicted Faults
69869     Line A Line B Line C  Line B Line C to Ground Fault
148983  Line B to Ground Fault         Line B to Ground Fault
70879   Line A to Line C fault         Line A to Line C fault
30362   Line C to Ground Fault         Line C to Ground Fault
126605  Line C to Ground Fault         Line C to Ground Fault
Classification Report:
                                      precision    recall  f1-score   support

                Line A Line B Line C       0.46      0.13      0.21      3229
Line A Line B Line C to Ground Fault       0.43      0.76      0.55      3141
       Line A Line B to Ground Fault       0.89      0.94      0.91      3211
       Line A Line C to Ground Fault       0.85      0.98      0.91      3183
              Line A to Ground Fault       0.93      0.92      0.92      3199
              Line A to Line B fault       0.87      0.84      0.85      3178
 

In [ ]:
def predict_fault_type_classification(input_data):
    # Preprocess input data (e.g., scale features)
    input_data_scaled = scaler.transform(input_data)
    # input_data_scaled.reshape(1, -1);
    print(input_data_scaled.shape);
    # Make predictions using the trained FNN model
    predictions_probabilities = model.predict(input_data_scaled)
    predicted_label = np.argmax(predictions_probabilities)
    print(predicted_label);

    # Decode predicted label to fault type
    predicted_fault_type = label_encoder.inverse_transform([predicted_label])[0]

    return predicted_fault_type

In [ ]:
def predict_fault_type_location(input_data):
    # Preprocess input data (e.g., scale features)
    input_data_scaled = scaler.transform(input_data)
    # input_data_scaled.reshape(1, -1);
    print(input_data_scaled.shape);
    # Make predictions using the trained FNN model
    predictions_probabilities = location_model.predict(input_data_scaled)
    predicted_label = np.argmax(predictions_probabilities)
    print(predicted_label);

    # Decode predicted label to fault type
    predicted_fault_type = location_label_encoder.inverse_transform([predicted_label])[0]

    return predicted_fault_type

In [103]:
# Take input manually for voltage and current measurements
# Va = float(input("Enter value for Va: "))
# Vb = float(input("Enter value for Vb: "))
# Vc = float(input("Enter value for Vc: "))
# Ia = float(input("Enter value for Ia: "))
# Ib = float(input("Enter value for Ib: "))
# Ic = float(input("Enter value for Ic: "))

# Convert input values to a list
input_values_1d = np.array([152.072382546,	-18.1590523791,	-28.096775352,	0.0496369354781,	1.21327365785,	-0.973767396711]);
input_values_2d = input_values_1d.reshape(1, -1);
# Predict fault type using the trained FNN model
predicted_fault_type = predict_fault_type_classification(input_values_2d)
predicted_fault_location = predict_fault_type_location(input_values_2d)

# Display predicted fault type
print("Predicted Fault Type:", predicted_fault_type)

# Display predicted fault type
print("Predicted Fault Location:", predicted_fault_location)

# Enter value for Va: 115.82
# Enter value for Vb: 23.333
# Enter value for Vc: -139.16
# Enter value for Ia: 0.0587
# Enter value for Ib: 0.04451
# Enter value for Ic: -0.103219



(1, 6)
1/1 [==============================] - 0s 23ms/step
2
(1, 6)
1/1 [==============================] - 0s 23ms/step
6
Predicted Fault Type: Line A Line B to Ground Fault
Predicted Fault Location: 100
